In [84]:
# Load packages
import time
import numpy as np
import pandas as pd
from numba import jit,float64
from scipy.optimize import minimize

In [56]:
# Load data
data = pd.read_csv('UnitaryData.csv')
pd_lag = np.array(data['d.p'])

# Calculate terciles for pd ratio
tercile_1 = np.quantile(pd_lag,1./3)
tercile_2 = np.quantile(pd_lag,2./3)

# Calculate indicator based on today's pd ratio
pd_lag_indicator = np.array([pd_lag <= tercile_1,(pd_lag <= tercile_2) & (pd_lag > tercile_1),pd_lag > tercile_2]).T

# Calculate indicator for tomorrow's pd ratio
pd_indicator = pd_lag_indicator[1:]

# Drop last row since we do not have tomorrow's pd ratio at that point
pd_lag_indicator = pd_lag_indicator[:-1]
X = np.array(data[['Rf','Rm-Rf','SMB','HML']])[:-1]
f = np.hstack((X * pd_lag_indicator[:,:1],X * pd_lag_indicator[:,1:2],X * pd_lag_indicator[:,2:3]))
g = np.array(data['log.RW'])[:-1]

## 7 Incorporating Conditional Information

### 7.1 Approach I
\begin{equation}
\min_{M\geq0}{\mathbb E}\left[Mg(X)\right]
\end{equation}
*subject to constraints:*
\begin{align*}
&{\mathbb E}\left[M \log M\right] \leq \kappa, \\
&{\mathbb E}\left[B^j M f(X)\right] = 0, \\
&{\mathbb E}\left[M\right] = 1.
\end{align*}


### 7.2 Approach II
\begin{equation}
\min_{M\geq0}{\mathbb E}\left[Mg(X)\right]
\end{equation}
*subject to constraints:*
\begin{align*}
&{\mathbb E}\left[M \log M\right] \leq \kappa, \\
&{\mathbb E}\left[B^j M {f(X)}\right] = 0, \\
&{\mathbb E}\left[B^j M-B^j\right] = 0.
\end{align*}

### 7.3 Approach III

#### Basic problem:

\begin{equation}
\min_{M\geq0}{\mathbb E}\left[Mg(X)\mid\mathfrak{J}\right]
\end{equation}
*subject to constraints:*
\begin{align*}
&{\mathbb E}\left[M \log M\mid\mathfrak{J}\right] \leq \kappa, \\
&{\mathbb E}\left[M  f(X)\mid\mathfrak{J}\right] = 0, \\
&{\mathbb E}\left[M\mid\mathfrak{J}\right] = 1.
\end{align*}

#### Dual problem:
For computational purposes, we solve the dual problem after minimizing over $M$.  

\begin{equation*}
\max_{\xi \ge 0, \hat{\lambda}}    - \xi \log {\mathbb E} \left[ \exp\left( - {\frac 1 {\xi}} \left[ g(X) + \hat{\lambda} \cdot f(X) \right] \right)\mid\mathfrak{F}\right]  -  \xi \kappa  
\end{equation*}

$\hat{\lambda}$ and $\xi$ are multipliers on the moment condition and relative entropy constraints. 

## 8 Intertemporal Divergence Constraints

### Proposition 8.6
Problem 8.4 can be solved by finding the solution to:

\begin{equation}
\epsilon = \min_\hat{\lambda}\mathbb E \left(\exp \left[-\frac{1}{\xi}g(X_1)+\hat{\lambda}\cdot f(X_1)\right]\left( \frac{e_1}{e_0}\right) \mid \mathfrak{F}_0\right)
\end{equation}

*where*
\begin{align*}
\mu &= -\xi \log \epsilon,\\
v_0 &= -\xi \log e_0.
\end{align*}

Denote $e_0^*$, $e_1^*$, $\hat{\lambda}^*$ as the solution to the above optimization problem. The implied solution for the probablity distortion is:

\begin{equation}
M_1^* = \frac{\exp \left[-\frac{1}{\xi}g(X_1)+\hat{\lambda}^*(Z_0)\cdot f(X_1)\right]e_1^*}{\epsilon^*e_0^*}
\end{equation}

In [57]:
def objective(λ):
    selector = pd_lag_indicator[:,state-1]
    term_1 = -g[selector]/ξ
    term_2 = f[:,(state-1)*4:state*4][selector]@λ
    term_3 = np.log(pd_indicator[selector]@e)
    x = term_1 + term_2 + term_3
    # use "max trick" to improve accuracy
    a = x.max()
    # log_E_exp(x)
    return np.log(np.sum(np.exp(x-a))) + a

def min_objective():
    model = minimize(objective, 
                     np.ones(4), 
                     method='L-BFGS-B',
                     tol=1e-10,
                     options={'maxiter': 1000})
    v = np.exp(model.fun)/np.sum(pd_lag_indicator[:,state-1])
    λ = model.x
    return v,λ

def iteration():
    # set global variables
    global state
    global e
    
    # initial error
    error = 1.
    # count times
    count = 0

    while error > 1e-10:
        if count == 0:
            # initial guess for e
            e = np.array([1,1,1])
            # placeholder for v
            v = np.zeros(3)   
            # placeholder for λ
            λ = np.zeros(12)
        for k in [1,2,3]:
            state = k
            v[state-1],λ[(state-1)*4:state*4] = min_objective()
        # update e and ϵ
        e_old = e
        ϵ = v[0]
        e = v/v[0]
        error = np.max(np.abs(e - e_old))
        count += 1
        
    return ϵ,e,λ,count

In [58]:
ξ = 1.

time_start = time.time() 
ϵ,e,λ,count = iteration()

print("--- %s seconds ---" % (round(time.time()-time_start,4)))
print("--- %s iterations ---" % count)
print("--- ϵ: %s ---" % ϵ)
print("--- e: %s ---" % e)
print("--- λ: %s ---" % λ)

--- 1.7294 seconds ---
--- 235 iterations ---
--- ϵ: 0.9662646500347174 ---
--- e: [1.      0.44192 0.19224] ---
--- λ: [ 1.20466  0.79534 -0.31077 -0.76063  1.42673  0.5733   0.66143 -5.9703
  2.68479 -0.68482 -3.4839  -7.80325] ---


In [68]:
# Calculate M
M = 1./ϵ * np.exp(-g/ξ+f@λ) * (pd_indicator@e) / (pd_lag_indicator@e)

# Check 1: E[M|state k] = 1
print("E[M|state 1] = %s " % np.mean(M[pd_lag_indicator[:,0]]))
print("E[M|state 2] = %s " % np.mean(M[pd_lag_indicator[:,1]]))
print("E[M|state 3] = %s " % np.mean(M[pd_lag_indicator[:,2]]))

E[M|state 1] = 0.9999999999954226 
E[M|state 2] = 0.9999999997898478 
E[M|state 3] = 0.9999999996547466 


In [69]:
# Calculate  conditional relative entropy
RE_1 = np.mean(M[pd_lag_indicator[:,0]]*np.log(M[pd_lag_indicator[:,0]]))
RE_2 = np.mean(M[pd_lag_indicator[:,1]]*np.log(M[pd_lag_indicator[:,1]]))
RE_3 = np.mean(M[pd_lag_indicator[:,2]]*np.log(M[pd_lag_indicator[:,2]]))

# Print conditional relative entropy
print("E[MlogM|state 1] = %s " % RE_1)
print("E[MlogM|state 2] = %s " % RE_2)
print("E[MlogM|state 3] = %s " % RE_3)

E[MlogM|state 1] = 0.012404221512381448 
E[MlogM|state 2] = 0.06706655850983517 
E[MlogM|state 3] = 0.16819195810069526 


In [92]:
# Calculate transition matrix under the distorted belief
P = np.zeros((3,3))
for i in [1,2,3]:
    for j in [1,2,3]:
        P[i-1,j-1] = np.mean(M[pd_lag_indicator[:,i-1]]*pd_indicator[pd_lag_indicator[:,i-1]][:,j-1])
# Print out the transition matrix
print("P_tilde:\n", P)
# P@np.ones(3).reshape(3,1)

P_tilde:
 [[0.97955 0.02045 0.     ]
 [0.08402 0.88102 0.03496]
 [0.      0.17515 0.82485]]


In [103]:
# Calculate the stationary distribution
A = P.T - np.eye(3)
A[-1] = np.ones(3)
B = np.zeros(3)
B[-1] = 1.
π = np.linalg.solve(A, B)
# π_check = np.linalg.matrix_power(P, 500)
print("π = %s" % π)

π = [0.77399 0.1884  0.03761]


In [107]:
# Calculate unconditional relative entropy
RE = np.array([RE_1,RE_2,RE_3]) @ π
print("E[MlogM] = %s" % RE)

E[MlogM] = 0.028561418675555502


In [118]:
# Implied moment bound, 1st approach: use \mu-\xi*RE
moment_bound = - 1./ξ * np.log(ϵ) - ξ*RE

# # Implied moment bound, 2nd approach: use E[Mg(X)|state]π
# Mg_1 = np.mean(M[pd_lag_indicator[:,0]]*g[pd_lag_indicator[:,0]])
# Mg_2 = np.mean(M[pd_lag_indicator[:,1]]*g[pd_lag_indicator[:,1]])
# Mg_3 = np.mean(M[pd_lag_indicator[:,2]]*g[pd_lag_indicator[:,2]])
# moment_bound_check = np.array([Mg_1,Mg_2,Mg_3]) @ π

print("E[Mg(X)] = %s" % moment_bound)

E[Mg(X)] = 0.005756098776325266
